In [4]:
import os
import pandas as pd
import numpy as np
import re
import selenium
from selenium import webdriver

import random
import time

from bs4 import BeautifulSoup
from urllib.request import urlopen

In [5]:
# Get suburb name and state from aus_suburb table.
def convert_to_acronym(state_var):
    if state_var == 'New South Wales':
        state_var = 'NSW'
    elif state_var == 'Victoria':
        state_var = 'Vic'
    elif state_var == 'Australian Capital Territory':
        state_var = 'ACT'
    elif state_var == 'Queensland':
        state_var = 'Qld'
    elif state_var == 'South Australia':
        state_var = 'SA'
    elif state_var == 'Northern Territory':
        state_var = 'NT'
    elif state_var == 'Western Australia':
        state_var = 'WA'
    elif state_var == 'Tasmania':
        state_var = 'Tas'
    return state_var

In [243]:
def find_abs_quick_stat_webpage(suburb, state):
    # Select the id box
    id_box = driver.find_element_by_class_name('gwt-SearchWidget-SuggestBox-Fade')

    # Send id information
    id_box.send_keys(suburb)

    # Time delay (minimum of 0.2 sec required)
    num = random.uniform(1.0,2.0)
    print(num)
    time.sleep(num)

    # Get pop-up table
    popup_table_id = driver.find_element_by_class_name('suggestPopupMiddleCenter')

    # Get suburb index from popup table
    rows = popup_table_id.find_elements_by_tag_name("tr")

    counter = -1
    var_check = 0
    for row in rows:
        counter = counter + 1
        popup_text = row.text
        if 'State Suburb (SSC)' in popup_text:
            if state in popup_text:
                popup_text = popup_text.replace('State Suburb (SSC)', '')
                popup_text = popup_text.split(',')[0].split('(')[0]
                popup_text = popup_text.strip()
                if suburb == popup_text:
                    var_check = 1
                    break
                    
    # If State Suburb doesn't exist for suburb in the given popup list,
    # then check for State Electoral Division.
    
    if var_check == 0:
        counter = -1
        for row in rows:
            counter = counter + 1
            popup_text = row.text
            if 'State Electoral Division (SED)' in popup_text:
                if state in popup_text:
                    popup_text = popup_text.replace('State Electoral Division (SED)', '')
                    popup_text = re.sub('[\(\[].*?[\)\]]', '', popup_text)
                    popup_text = popup_text.split(',')[0].split('(')[0]
                    popup_text = popup_text.strip()
                    if suburb == popup_text:
                        print(popup_text)
                        break
                
    # Get Suburb gwt-uid from popup table 
    IDs = popup_table_id.find_elements_by_tag_name('td')
    suburb_uid = IDs[counter].get_attribute('id')

    # Find suburb by id
    popup_item = driver.find_element_by_id(suburb_uid)

    # Select suburb
    popup_item.click()

    # Find GO button
    go_button = driver.find_element_by_class_name('gwt-SearchWidget-Button')

    # Click GO
    go_button.click()

In [7]:
# Function for getting the tables on html page
def get_tables(soup_html, id_content):
    content_tables = soup_html.find('div', attrs={'class':'content', 'id':id_content})
    return content_tables.find_all('table')

In [8]:
# Function for getting data from the tables and store in a dictionary
def scrape_stats(tables):
    data = {}
        
    for table in tables:
        
        table_rows = table.find_all('tr')
        table_name = table_rows[0].find('a', attrs={'class':'dictionaryLink'}).text        

        for i in range(1, len(table_rows)):
            
            try:
                row_label = table_rows[i].find('th', attrs={'class':'firstCol','scope':'row'}).text
            except(AttributeError):
                row_label = table_rows[i].find('th', attrs={'class':'firstCol'}).text

            count = table_rows[i].find_all('td')[0].text
            percentage = table_rows[i].find_all('td')[1].text

            data['{}_{}_{}'.format(table_name, row_label, 'count')] = count
            data['{}_{}_{}'.format(table_name, row_label, 'percentage')] = percentage
            
    return data

In [9]:
# Function combining get_tables and scrape_data functions
def get_stats(soup_html, id_content):
    tables = get_tables(soup_html, id_content)
    table_data = scrape_stats(tables)
    return table_data

In [10]:
# Function for getting quick stat tables
def get_qs_tables(soup_html):
    quick_stat_tables = soup_html.find('div', attrs={'id':'summaryTableAP'})
    return quick_stat_tables.find_all('table')

In [11]:
# Function for scraping quick stat tables
def scrape_qs(qs_tables):
    data = {}

    for qs_table in qs_tables:

        qs_table_rows = qs_table.find_all('tr')
        
        # Table labels and values for first row
        qs_table_name = qs_table_rows[0].find('th', attrs={'class':'addBold', 'scope':'row'}).text        
        qs_row_label = qs_table_name
        qs_row_value = qs_table_rows[0].find('td', attrs={'class':'summaryData'}).text

        data['{}_{}_{}'.format(qs_table_name, qs_row_label, 'count')] = qs_row_value
        
        # Table labels and values for second row and onwards
        for i in range(1, len(qs_table_rows)):
            
            qs_row_label = qs_table_rows[i].find('th', attrs={'scope':'row'}).text

            try:            
                qs_row_value = qs_table_rows[i].find('td', attrs={'class':'summaryData'}).text                
                if "%" in qs_row_value:            
                    data['{}_{}_{}'.format(qs_table_name, qs_row_label, 'percentage')] = qs_row_value        
                else:                          
                    data['{}_{}_{}'.format(qs_table_name, qs_row_label, 'count')] = qs_row_value                
            except(AttributeError):            
                pass
    
    # Change row label names
    data['Families_Avg children per family_for families with children_count'] = data.pop('Families_for families with children_count')
    data['Families_Avg children per family_for all families_count'] = data.pop('Families_for all families_count')
    
    data['All private dwellings_Average number of people per household_count'] = data.pop('All private dwellings_Average people per household_count')

    return data    

In [12]:
# Function combining get_tables and scrape_data functions
def get_qs_stats(soup_html):
    qs_tables = get_qs_tables(soup_html)
    qs_table_data = scrape_qs(qs_tables)
    return qs_table_data

### Get demographic data from Australian Bureau of Statistics

In [53]:
# Load file
aus_suburbs = pd.read_csv(os.path.join('aus_suburbs.csv'), index_col=0)

In [238]:
# Using Chrome to access web
driver = webdriver.Chrome()

driver.get('https://www.abs.gov.au/websitedbs/D3310114.nsf/Home/2016%20QuickStats')

In [248]:
# Using Chrome to access web
driver = webdriver.Chrome()

driver.get('https://www.abs.gov.au/websitedbs/D3310114.nsf/Home/2016%20QuickStats')

# ABS_data = {}

# for index,row in aus_suburbs.iterrows():
for i in range(1931, aus_suburbs.shape[0]):
    # Get suburb and state names from aus_suburbs table
#     suburb = row.Suburb
#     state = row.State
    suburb = aus_suburbs.iloc[i].Suburb
    state = aus_suburbs.iloc[i].State
    state = convert_to_acronym(state)

    print('{} {}'.format(suburb, state))

    while True:
        # Open the website
        driver.get('https://www.abs.gov.au/websitedbs/D3310114.nsf/Home/2016%20QuickStats')

        # Select census year
        year = '2016'
        census_cycle = driver.find_element_by_class_name('gwt-ListBox')
        census_year = census_cycle.find_element_by_xpath('//option[@value={}]'.format(year))
        census_year.click()

        # Go to suburb's ABS url page
        try:
            find_abs_quick_stat_webpage(suburb, state)
        except(Exception) as e:
            print('{} {} has no ABS page'.format(suburb, state))
            break
            
        # Get suburb's QuickStat URL
        url = driver.current_url

        # Get html page, title, and state from URL
        soup = BeautifulSoup(urlopen(url), "html.parser")
        
        page_state = soup.find('span', attrs={'class':'geoType'}).text
        page_state = page_state[12:]
        page_state = page_state.replace(' | State Suburbs','').replace(' | State Electoral Divisions','')
        page_state = convert_to_acronym(page_state)

        page_title = soup.title.text
        page_title = page_title.replace('2016 Census QuickStats: ', '')
        page_title = re.sub('[\(\[].*?[\)\]]', '', page_title)
        page_title = page_title.strip()

        print(soup.title.text)
        if (suburb == page_title) and (state == page_state):
            break
        else:
            print('Wrong page')
            continue

    # Scrape data from tables
    try:
        people = get_stats(soup, 'peopleContent')
        family = get_stats(soup, 'familyContent')
        dwelling = get_stats(soup, 'dwellingContent')
        ingp = get_stats(soup, 'INGPContent')

        # Merge datasets into one dictionary
        suburb_stats = {**people, **family, **dwelling, **ingp}

        # Add state as subkey
        suburb_stats['State'] = state

    except(AttributeError, NameError) as e:    
        # Scrape data that only have a quick stat table
        try:
            suburb_stats = get_qs_stats(soup)
            suburb_stats['State'] = state

        # Suburb does not contain any stats
        except(AttributeError):
            print(suburb)
            suburb_stats = {}

    # Add suburb dictionary to main dictionary
    ABS_data['{}_{}'.format(suburb,state)] = suburb_stats

Richmond Vic
1.7696191498063762
Richmond
2016 Census QuickStats: Richmond (Northern Metropolitan)
Richmond NSW
1.2590363988965012
2016 Census QuickStats: Richmond (NSW)
Richmond Lowlands NSW
1.3244471122841812
2016 Census QuickStats: Richmond Lowlands
Ridgehaven SA
1.1057350614385955
2016 Census QuickStats: Ridgehaven
Ridgeway Tas
1.6494747376461278
2016 Census QuickStats: Ridgeway
Ridgewood WA
1.6266758605044809
2016 Census QuickStats: Ridgewood (WA)
Ridleyton SA
1.030489228158183
2016 Census QuickStats: Ridleyton
Ringwood Vic
1.588044445134704
2016 Census QuickStats: Ringwood (Vic.)
Ringwood East Vic
1.8852261604183755
2016 Census QuickStats: Ringwood East
Ringwood North Vic
1.6509871893293762
2016 Census QuickStats: Ringwood North
Ripley Qld
1.9589049458089758
2016 Census QuickStats: Ripley
Ripponlea Vic
1.1517594187981544
2016 Census QuickStats: Ripponlea
Risdon Tas
1.7448288767468128
2016 Census QuickStats: Risdon
Risdon Vale Tas
1.2021928594953828
2016 Census QuickStats: Risdon V

1.1475178979462621
2016 Census QuickStats: Scoresby
Scullin ACT
1.7280275578378674
2016 Census QuickStats: Scullin
Seabrook Vic
1.859096305913611
2016 Census QuickStats: Seabrook
Seacliff SA
1.8121293985321167
2016 Census QuickStats: Seacliff
Seacliff Park SA
1.8233166672090755
2016 Census QuickStats: Seacliff Park
Seacombe Gardens SA
1.8189059018378488
2016 Census QuickStats: Seacombe Gardens
Seacombe Heights SA
1.7310171670432997
2016 Census QuickStats: Seacombe Heights
Seaford Vic
1.1809507808492121
2016 Census QuickStats: Seaford (Vic.)
Seaford SA
1.7593937978953316
2016 Census QuickStats: Seaford (SA)
Seaford Heights SA
1.9476883067057842
2016 Census QuickStats: Seaford Heights
Seaford Meadows SA
1.6786034608014029
2016 Census QuickStats: Seaford Meadows
Seaford Rise SA
1.0477523551655978
2016 Census QuickStats: Seaford Rise
Seaforth NSW
1.242355990238297
Seaforth NSW has no ABS page
Seaholme Vic
1.1921830120056292
2016 Census QuickStats: Seaholme
Seaton SA
1.4127539836013927
2016

1.2357303618137987
2016 Census QuickStats: St Clair (SA)
St Clair NSW
1.8141564210381396
2016 Census QuickStats: St Clair (Penrith - NSW)
St Georges SA
1.1624118516369124
2016 Census QuickStats: St Georges
St Helena Vic
1.9298476300220684
2016 Census QuickStats: St Helena
St Helens Park NSW
1.6001637737192462
2016 Census QuickStats: St Helens Park
St Ives NSW
1.5395952521799852
2016 Census QuickStats: St Ives (NSW)
St Ives Chase NSW
1.3218487902384983
2016 Census QuickStats: St Ives Chase
St James WA
1.8718037792464868
2016 Census QuickStats: St James (WA)
St Johns Park NSW
1.6242758344146044
2016 Census QuickStats: St Johns Park
St Kilda SA
1.396354302906209
2016 Census QuickStats: St Andrews (Vic.)
Wrong page
1.00238862489453
2016 Census QuickStats: St Kilda (SA)
St Kilda Vic
1.6813631615704894
2016 Census QuickStats: St Kilda (Vic.)
St Kilda East Vic
1.830117350428035
2016 Census QuickStats: St Kilda East
St Kilda West Vic
1.6582297533317902
2016 Census QuickStats: St Kilda West
St 

1.573571679489478
2016 Census QuickStats: Mount Thorley
Wrong page
1.2408029339709834
2016 Census QuickStats: Thornlands
Thornleigh NSW
1.2665932673258622
2016 Census QuickStats: Thornleigh
Thornlie WA
1.737910138456006
2016 Census QuickStats: Thornlie
Three Bridges Vic
1.2771845696848114
2016 Census QuickStats: Three Bridges
Tingalpa Qld
1.0583823710869962
2016 Census QuickStats: Tingalpa
Tivendale NT
1.0502741504892557
2016 Census QuickStats: Tivendale
Tivendale
Tivoli Qld
1.3010402481128298
2016 Census QuickStats: Tivoli
Tiwi NT
1.1272101497707308
2016 Census QuickStats: Tiwi
Tolmans Hill Tas
1.092153319491981
2016 Census QuickStats: Tolmans Hill
Tonimbuk Vic
1.1263956096956416
2016 Census QuickStats: Tonimbuk
Toolern Vale Vic
1.700777324355435
2016 Census QuickStats: Toolern Vale
Toongabbie NSW
1.529786826273806
2016 Census QuickStats: Toongabbie (NSW)
Tooradin Vic
1.8664577017545696
2016 Census QuickStats: Tooradin
Toorak Vic
1.4960061401044427
2016 Census QuickStats: Toorak
Toora

1.9238071011593123
2016 Census QuickStats: Waterfall
Waterfall Gully SA
1.0408868407214489
2016 Census QuickStats: Waterfall Gully
Waterford Qld
1.4695362433191583
2016 Census QuickStats: Waterford (Qld)
Waterford WA
1.6855572042358213
2016 Census QuickStats: Manning - Waterford
Wrong page
1.9191319810699017
2016 Census QuickStats: Waterford (WA)
Waterford West Qld
1.175821773145055
2016 Census QuickStats: Waterford West
Waterloo NSW
1.93792822474605
2016 Census QuickStats: Waterloo (NSW)
Waterloo Corner SA
1.7988696672283133
2016 Census QuickStats: Waterloo Corner
Watermans Bay WA
1.8687592542659446
2016 Census QuickStats: Watermans Bay
Waterways Vic
1.2018446808042635
2016 Census QuickStats: Waterways
Watson ACT
1.825299764340782
2016 Census QuickStats: Watson
Watsonia Vic
1.3257165067041021
2016 Census QuickStats: Watsonia
Watsonia North Vic
1.3110595745239833
2016 Census QuickStats: Watsonia North
Watsons Bay NSW
1.282608183250074
2016 Census QuickStats: Watsons Bay
Watsons Creek V

2016 Census QuickStats: Winthrop
Wisemans Ferry NSW
1.454892770329755
2016 Census QuickStats: Wisemans Ferry
Wishart Qld
1.2125518746918211
2016 Census QuickStats: Wishart (Qld)
Wishart NT
1.9955942167370786
2016 Census QuickStats: Wishart (NT)
Wishart
Wolffdene Qld
1.9869337835834733
2016 Census QuickStats: Wolffdene
Wollert Vic
1.3798956907708537
2016 Census QuickStats: Wollert
Wolli Creek NSW
1.7881437325644083
2016 Census QuickStats: Wolli Creek
Wollstonecraft NSW
1.1087299526983918
2016 Census QuickStats: Wollstonecraft
Wonga Park Vic
1.2705115171888142
2016 Census QuickStats: Wonga Park
Woodbine NSW
1.8332835906852556
2016 Census QuickStats: Woodbine (NSW)
Woodbridge WA
1.8426463708314402
2016 Census QuickStats: Woodbridge (WA)
Woodcroft NSW
1.6207723132443852
2016 Census QuickStats: Woodcroft (NSW)
Woodcroft SA
1.2129775742934061
2016 Census QuickStats: Woodcroft (SA)
Woodend Qld
1.7484209023241961
2016 Census QuickStats: Woodburn (Shoalhaven - NSW)
Wrong page
1.4350954827635702

#### Convert dictionary to table


In [251]:
df = pd.DataFrame(ABS_data).T

df.index.name = 'Suburb'
df.reset_index(inplace=True)

# Move 'State' column to the front
cols = list(df)
cols.insert(1, cols.pop(cols.index('State')))
df = df.loc[:, cols]

In [256]:
df.head()

,Suburb,State,Age_0-4 years_count,Age_0-4 years_percentage,Age_10-14 years_count,Age_10-14 years_percentage,Age_15-19 years_count,Age_15-19 years_percentage,Age_20-24 years_count,Age_20-24 years_percentage,...,"Unpaid domestic work, number of hours_Less than 5 hours per week_count","Unpaid domestic work, number of hours_Less than 5 hours per week_percentage",Unpaid work_Cared for child/children (last two weeks)_count,Unpaid work_Cared for child/children (last two weeks)_percentage,Unpaid work_Did unpaid domestic work (last week)_count,Unpaid work_Did unpaid domestic work (last week)_percentage,Unpaid work_Did voluntary work through an organisation or group (last 12 months)_count,Unpaid work_Did voluntary work through an organisation or group (last 12 months)_percentage,Unpaid work_Provided unpaid assistance to a person with a disability (last two weeks)_count,Unpaid work_Provided unpaid assistance to a person with a disability (last two weeks)_percentage
0,Abbotsbury,NSW,214.0,5.0,279.0,6.6,347.0,8.2,377.0,8.9,...,880.0,25.1,884.0,25.2,2346.0,67.0,439.0,12.5,507.0,14.5
1,Abbotsford,NSW,306.0,5.7,239.0,4.5,225.0,4.2,202.0,3.8,...,1192.0,26.2,1233.0,27.1,3267.0,71.8,831.0,18.3,606.0,13.3
2,Abbotsford,Vic,319.0,3.9,153.0,1.9,210.0,2.6,760.0,9.3,...,2688.0,35.7,1132.0,15.0,5489.0,72.8,1506.0,20.0,563.0,7.5
3,Aberfeldie,Vic,179.0,4.6,291.0,7.5,348.0,8.9,284.0,7.3,...,891.0,28.1,921.0,29.0,2404.0,75.8,759.0,24.0,476.0,15.0
4,Aberfoyle Park,SA,587.0,5.3,677.0,6.1,846.0,7.7,691.0,6.3,...,2205.0,24.2,2775.0,30.5,7181.0,79.0,2151.0,23.7,1167.0,12.8


#### Clean data

In [254]:
# Remove characters from columns
for column in df:
    df[column] = df[column].str.replace(',', '')
    df[column] = df[column].str.replace('$', '')
    df[column] = df[column].str.replace('%', '')
    df[column] = df[column].replace('--', np.nan)
        
# Convert string to float
for column in df:
    if ('count' in column) or ('percentage' in column):
        df[column] = pd.to_numeric(df[column])

In [255]:
# Remove state title from suburb column
df['Suburb'] = df['Suburb'].str.split('_')
df['Suburb'] = df.Suburb.map(lambda x: x[0])

In [257]:
# Fill median age empty rows
df['Age_Median age_count'] = df.apply(
    lambda row: row['People_Median age_count'] if np.isnan(row['Age_Median age_count'])
                                                           else row['Age_Median age_count'], axis=1
)

df['People_People_count'] = df.apply(
    lambda row: row['People_Male_count'] + row['People_Female_count'] if np.isnan(row['People_People_count'])
                                                           else row['People_People_count'], axis=1
)

In [258]:
# Drop columns
columns = ['People_Median age_count', 'Age_Median age_percentage', 'Rent weekly payments_Median rent_percentage']
df.drop(columns, inplace=True, axis=1)

In [260]:
# Rename columns
df.rename({'Age_Median age_count':'Age_Median age',
           'Rent weekly payments_Median rent_count':'Rent weekly payments_Median rent'}, axis=1, inplace=True)

#### Save dataframe as csv

In [275]:
df.to_csv('ABS_2016_data.csv')